### final

In [ ]:
### 0. import
import psycopg2
import numpy  as np
import pandas as pd
from datetime import datetime

# display option (float 자리수)
pd.options.display.float_format = lambda x: f'{x:.4f}'


### 1. 데이터 가져오기
# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

# pnu별 상업용 부동산 면적, 매출액
sql = '''
select t1.pnu, t1.emd_cd, t1.ftc_cate2_cd, t1.sales, t2.avg_area
from (
	select substring (bd_mgt_sn, 0, 20) as pnu, pred_slng_amt as sales, ftc_cate2_cd, emd_cd||'00' as emd_cd
	  from m1.kt_bldg_sales
	 where base_ym = '202209'
	)as t1
left join (
	select sigungu_cd||bjdong_cd||case when plat_gb_cd = '0' then '1' else plat_gb_cd end||bun||ji as pnu, avg(tot_area) as avg_area, substring(pnu,0,11) as emd_cd
	  from m1.bld_rgst
	 where main_purps_cd_nm not  like '%주택%'
       and base_ym = '202209'
	 group by sigungu_cd||bjdong_cd||case when plat_gb_cd = '0' then '1' else plat_gb_cd end||bun||ji
	) t2
  on t1.emd_cd = t2.emd_cd 
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df = pd.DataFrame(rows)
df.columns = ['pnu','emd_cd','industry_cd','sales','area_avg']

In [ ]:
df

In [ ]:
# 매출액 / 면적
df['sales_per_area'] = (df['sales'] / df['area_avg'])

# inf 값 제거후 컬럼 정리
df = df[~(df['sales_per_area']==np.inf)]
df = df[['pnu','emd_cd','sales_per_area','industry_cd']] # ftc_cate2_cd

# 작업일(업데이트일) 및 작업자 추가
df['create_at'] = datetime.today().strftime("%Y-%m-%d")
df['update_at'] = datetime.today().strftime("%Y-%m-%d")
df['work_user'] = 'du.Park'


### 2. DB INSERT
import datetime

table = 'm2.datadam_sales_per_area'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))

In [ ]:
df

In [ ]:
df['sales_per_area'].median()

###### ver1

In [19]:
### 0. import
import psycopg2
import numpy  as np
import pandas as pd
from datetime import datetime

# display option (float 자리수)
pd.options.display.float_format = lambda x: f'{x:.6f}'


### 1. 데이터 가져오기
# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

# 법정동별 상가 면적
sql = '''
select sigungu_cd||bjdong_cd as pnu, count(mgm_bldrgst_pk) as total_count
  from m1.bld_rgst
 where main_purps_cd_nm not in 
    (
    select distinct main_purps_cd_nm
    from m1.bld_rgst
    where main_purps_cd_nm like '%주택%'
    )
group by sigungu_cd||bjdong_cd
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_bld_rgst = pd.DataFrame(rows)
df_bld_rgst.columns = ['emd_cd','total_count']

# 법정동별 매출액
sql = '''
select lgl_dong_cd||'00' as emd_cd, avg(avg_biz_income) as sales_avg
  from m1.kcb_soho	 
 group by lgl_dong_cd
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_kcb_soho = pd.DataFrame(rows)
df_kcb_soho.columns = ['emd_cd','sales_avg']
df_kcb_soho['sales_avg'] = (df_kcb_soho['sales_avg']*1000).astype(float)


# 데이터 프레임 병합
df = pd.merge(df_kcb_soho, df_bld_rgst, on='emd_cd', how='left')

# 인구가 0명, null인 지역 제외
df = df[~df['total_count'].isnull()]

# 매출액 / 면적
df['sales_per_area'] = (df['sales_avg'] / df['total_count'])
df = df[['emd_cd','sales_per_area']]
df['base_ym'] = datetime.today().strftime("%Y%m")


### 2. DB INSERT
import datetime

table = 'm2.datadam_sales_per_area'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
     # drop & create
    cursor.execute(
    """
    drop table if exists m2.datadam_sales_per_area;
    create table m2.datadam_sales_per_area (    
    emd_cd varchar(10)
    , sales_per_area double precision
    , base_ym varchar(6)
    )
    distkey(emd_cd)
    compound sortkey(base_ym);
    commit;
    """
    )
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))

2022-10-30 20:29:02.684279  :  0
3650


In [15]:
df

,emd_cd,sales_avg,total_count
0,1111010400,68462000.000000,39.000000
1,1111010700,148435000.000000,42.000000
2,1111010800,64937000.000000,115.000000
3,1111010900,43201000.000000,17.000000
4,1111011000,107058000.000000,78.000000
...,...,...,...
4934,4617012900,19364000.000000,46.000000
4935,4219011000,14371000.000000,15.000000
4936,4518011100,84256000.000000,60.000000
4937,3611011700,36791000.000000,19.000000


In [18]:
df.describe()

,sales_per_area
count,3650.000000
mean,3459872.963816
std,15581744.463154
min,0.000000
25%,238977.037037
50%,494613.013699
75%,1257581.140351
max,439616000.000000


###### ver2 (kt 업종 데이터 가져와서 병합하면 메모리터짐)

In [1]:
### 0. import
import psycopg2
import numpy  as np
import pandas as pd
from datetime import datetime

# display option (float 자리수)
pd.options.display.float_format = lambda x: f'{x:.4f}'


### 1. 데이터 가져오기
# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

# pnu별 상업용 부동산 평균 연면적
sql = '''
select sigungu_cd||bjdong_cd||case when plat_gb_cd = '1' then '2' else '1' end||bun||ji as pnu, avg(tot_area) as avg_area, substring(pnu,0,11)
  from m1.bld_rgst
 where main_purps_cd_nm not in 
    (
    select distinct main_purps_cd_nm
    from m1.bld_rgst
    where main_purps_cd_nm like '%주택%'
    )
 group by sigungu_cd||bjdong_cd||case when plat_gb_cd = '1' then '2' else '1' end||bun||ji
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_bld_rgst = pd.DataFrame(rows)
df_bld_rgst.columns = ['pnu','area_avg','emd_cd']

# PNU별 매출액
sql = '''
select substring (bd_mgt_sn, 0, 20) as pnu, pred_slng_amt as sales, ftc_cate2_cd, emd_cd||'00'
  from m1.kt_bldg_sales
 where base_ym = '202209'
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_kt_bldg_sales = pd.DataFrame(rows)
df_kt_bldg_sales.columns = ['pnu','sales','industry_cd','emd_cd']

In [6]:
df_bld_rgst

,pnu,area_avg,emd_cd
0,1111010100100010000,449.8300,1111010100
1,1111010100100050000,397.6000,1111010100
2,1111010100100070003,81.9000,1111010100
3,1111010100100300007,136.3400,1111010100
4,1111010100100300013,998.3000,1111010100
...,...,...,...
2077137,4211040028106890012,60.0000,4211040028
2077138,4885039026202880000,78.0000,4885039026
2077139,4888025027111860001,47.7900,4888025027
2077140,4613015000108920000,66.9600,4613015000


In [7]:
df_kt_bldg_sales

,pnu,sales,industry_cd,emd_cd
0,3114010100102820002,5740478,B14,3114010100
1,3114010100102820002,100000000,B21,3114010100
2,3114010100102820002,1323405,B20,3114010100
3,3114010100102820002,12688502,A07,3114010100
4,3114010100102820002,8821770,A02,3114010100
...,...,...,...,...
58555804,5013032026117900000,14151894,C01,5013032000
58555805,5013032026117900000,8786831,B10,5013032000
58555806,5013032026117900000,2477928,A05,5013032000
58555807,5013032026117900000,26818739,A10,5013032000


In [12]:
# 데이터 프레임 병합
df_ = pd.merge(df_kt_bldg_sales, df_bld_rgst, on='emd_cd', how='left')

MemoryError: Unable to allocate 218. GiB for an array with shape (29211534645,) and data type int64

In [1]:
# null 제거
df = df[~df['area_avg'].isnull()]

# 매출액 / 면적
df['sales_per_area'] = (df['sales'] / df['area_avg'])

# inf 값 제거후 컬럼 정리
df = df[~(df['sales_per_area']==np.inf)]
df = df[['pnu','emd_cd','sales_per_area','industry_cd']]

# 작업일(업데이트일) 및 작업자 추가
df['create_at'] = datetime.today().strftime("%Y-%m-%d")
df['update_at'] = datetime.today().strftime("%Y-%m-%d")
df['work_user'] = 'du.Park'

KeyError: 'emd_cd'

In [ ]:
### 2. DB INSERT
import datetime

table = 'm2.datadam_sales_per_area'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))

In [ ]:
df['sales_per_area'].median()

In [ ]:
df.describe()